In [1]:
import os
import numpy as np

# 1. Get the current working directory
current_dir = os.getcwd()

# Load the memory-mapped tensor
super_tensor_file = os.path.join(current_dir, "super_emissions_tensor.npy")
molecule_types = ["CH4", "CO2", "CO2bio", "GWP", "N2O"]
data_types = ["emi", "flx"]

# Load the super tensor
super_tensor_shape = (5, 2, 288, 1800, 3600)
super_tensor = np.memmap(super_tensor_file, dtype='float32', mode='r', shape=super_tensor_shape)

In [2]:
super_tensor.shape

(5, 2, 288, 1800, 3600)

In [3]:
import numpy as np

# Assuming you have already loaded your memory-mapped tensor as:
# super_tensor = np.memmap(super_tensor_file, dtype='float32', mode='r', shape=(5, 2, 288, 1800, 3600))

# Reshape the last two dimensions so that:
# 1800 -> (600, 3) and 3600 -> (1200, 3)
reshaped = super_tensor.reshape(5, 2, 288, 600, 3, 1200, 3)

# Compute the mean over the 3x3 blocks (axis 4 and axis 6)
kernelized_tensor = reshaped.mean(axis=(4, 6))

print(kernelized_tensor.shape)  # Expected shape: (5, 2, 288, 600, 1200)

(5, 2, 288, 600, 1200)


In [4]:
np.save("kernelized_tensor.npy", kernelized_tensor)

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# --- Data Dimensions ---
num_molecules = 5         # CH4, CO2, CO2bio, GWP, N2O
num_data_types = 2        # Emission, Flux (we use Emission only: index 0)
num_time = 288            # 12 months * 24 years
lat = 600                 # kernelized latitude dimension
lon = 1200                # kernelized longitude dimension

output_dim = num_molecules * lat * lon  # Total output size per time sample

# --- Use preloaded tensor ---
# Make sure that 'kernelized_tensor' is already loaded in your environment
# and has shape (5, 2, 288, 600, 1200)
data = kernelized_tensor

# Select emissions only (data type index 0), resulting in shape: (5, 288, 600, 1200)
data_emissions = data[:, 0, :, :, :]

# Rearrange dimensions so that time is the first dimension: (288, 5, 600, 1200)
emissions_data = np.transpose(data_emissions, (1, 0, 2, 3))

# Reshape Y to be (num_time, 5, 600, 1200)
Y = emissions_data.reshape(num_time, num_molecules, lat, lon)

# --- Prepare Time Features and Train/Test Split ---
# Normalize the month index to [0,1] to help stabilize training.
X = (np.arange(num_time).reshape(-1, 1).astype(np.float32)) / (num_time - 1)

# Define train and test splits:
# Train on the first 48 months (first 4 years)
train_time = 48
# Test on the 12 months directly after the training period (months 48 to 59)
test_time = 12

X_train = X[:train_time]
X_test = X[train_time:train_time + test_time]

Y_train = Y[:train_time]
Y_test = Y[train_time:train_time + test_time]

print("Training samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])
print("Output shape per sample:", Y_train.shape[1:])  # Should print (5, 600, 1200)

# --- Custom Callback for Progress Monitoring ---
class ProgressCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        loss = logs.get('loss')
        print(f"Epoch {epoch+1:03d}: Loss = {loss:.6f}")

progress_callback = ProgressCallback()

# --- Build the MLP Model ---
model = Sequential([
    Input(shape=(1,)),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    # Dense layer outputs a flattened vector of length output_dim
    Dense(output_dim, activation='linear'),
    # Reshape to the desired output shape: (5, 600, 1200)
    Reshape((num_molecules, lat, lon))
])

# Lower the learning rate to help avoid numerical instability
model.compile(optimizer=Adam(learning_rate=1e-5), loss='mse')
model.summary()

# --- Train the Model ---
history = model.fit(
    X_train, Y_train, 
    epochs=50, 
    batch_size=8, 
    verbose=0,  # Turn off default verbose output; our callback prints progress
    callbacks=[progress_callback]
)

# --- Plot Training Loss History ---
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], marker='o')
plt.xlabel('Epoch')
plt.ylabel('Training Loss (MSE)')
plt.title('Training Loss Progress')
plt.grid(True)
plt.show()

# --- Evaluate the Model ---
Y_pred = model.predict(X_test)
test_mse = mean_squared_error(Y_test.flatten(), Y_pred.flatten())
print("Test MSE:", test_mse)

# --- Visualize a Sample Prediction ---
# Visualize the ground truth and prediction for the first test sample of the first molecule.
sample_idx = 0  # first test sample
molecule_idx = 0  # first molecule

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(Y_test[sample_idx][molecule_idx], cmap='viridis')
plt.title(f'Ground Truth (Molecule {molecule_idx})')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(Y_pred[sample_idx][molecule_idx], cmap='viridis')
plt.title(f'Prediction (Molecule {molecule_idx})')
plt.colorbar()

plt.tight_layout()
plt.show()

Training samples: 48
Test samples: 12
Output shape per sample: (5, 600, 1200)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3600000)        │   464,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 5, 600, 1200)   │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 464,408,448 (1.73 GB)

 Trainable params: 464,408,448 (1.73 GB)

 Non-trainable params: 0 (0.00 B)

Epoch 001: Loss = nan
Epoch 002: Loss = nan


KeyboardInterrupt: 